# Taylor–Green Benchmark (Classical vs NMSI–π*–γ_diss–e*)

This notebook runs a **2D pseudo-spectral Taylor–Green** surrogate twice:

- **Classical NSE**
- **Augmented (NMSI)** with π* cyclic forcing, γ_diss spectral window, and e* exponential damping

It produces time series for **Energy** $E(t)$, **Enstrophy** $\Omega(t)$, and **max vorticity** $\|\omega\|_\infty$, and saves results to NPZ files for easy comparison.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Grid and spectral setup (2D periodic)
Nx = Ny = 64
Lx = Ly = 2*np.pi
dx, dy = Lx/Nx, Ly/Ny
x = np.arange(Nx)*dx
y = np.arange(Ny)*dy
X, Y = np.meshgrid(x, y, indexing='ij')

kx = np.fft.fftfreq(Nx, d=dx/(2*np.pi))
ky = np.fft.fftfreq(Ny, d=dy/(2*np.pi))
KX, KY = np.meshgrid(kx, ky, indexing='ij')

def project_div_free(u):
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    kxr = KX[:, :uhx.shape[1]]; kyr = KY[:, :uhx.shape[1]]
    K2r = (kxr**2 + kyr**2)
    K2r[0,0] = 1.0
    div_hat = kxr*uhx + kyr*uhy
    uhx -= kxr*div_hat/K2r
    uhy -= kyr*div_hat/K2r
    ux = np.fft.irfftn(uhx, s=(Nx,Ny), axes=(0,1))
    uy = np.fft.irfftn(uhy, s=(Nx,Ny), axes=(0,1))
    return np.stack([ux, uy], axis=-1)

def grad(u):
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    kxr = KX[:, :uhx.shape[1]]; kyr = KY[:, :uhx.shape[1]]
    ux_x = np.fft.irfftn(1j*kxr*uhx, s=(Nx,Ny), axes=(0,1))
    ux_y = np.fft.irfftn(1j*kyr*uhx, s=(Nx,Ny), axes=(0,1))
    uy_x = np.fft.irfftn(1j*kxr*uhy, s=(Nx,Ny), axes=(0,1))
    uy_y = np.fft.irfftn(1j*kyr*uhy, s=(Nx,Ny), axes=(0,1))
    return ux_x, ux_y, uy_x, uy_y

def nonlinear_term(u):
    ux_x, ux_y, uy_x, uy_y = grad(u)
    advx = u[...,0]*ux_x + u[...,1]*ux_y
    advy = u[...,0]*uy_x + u[...,1]*uy_y
    return -np.stack([advx, advy], axis=-1)

def laplacian(u):
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    k2r = (KX[:, :uhx.shape[1]]**2 + KY[:, :uhx.shape[1]]**2)
    lx = np.fft.irfftn(-k2r*uhx, s=(Nx,Ny), axes=(0,1))
    ly = np.fft.irfftn(-k2r*uhy, s=(Nx,Ny), axes=(0,1))
    return np.stack([lx, ly], axis=-1)

def vorticity(u):
    ux_x, ux_y, uy_x, uy_y = grad(u)
    return uy_x - ux_y

def energy(u):    return 0.5*np.mean((u**2).sum(-1))
def enstrophy(u): return np.mean(vorticity(u)**2)
def wmax(u):      return np.abs(vorticity(u)).max()

# NMSI operators
A_pi, omega_pi, phi = 0.2, 2.0, 0.0
lam_e, alpha_e      = 0.4, 0.2
gamma0              = 0.6
nu                  = 1e-3

def Z_window_k(shape):
    # Construct kmag on rfft grid
    nkr = shape[1]
    kxr = KX[:, :nkr]; kyr = KY[:, :nkr]
    kmag = np.sqrt(kxr**2 + kyr**2)
    kmax = kmag.max()
    return ((kmag >= 0.6*kmax) & (kmag <= 0.9*kmax)).astype(float)

def step(u, t, dt, augmented=True):
    NL = nonlinear_term(u)
    visc = nu*laplacian(u)

    if augmented:
        Fpi = A_pi*np.sin(omega_pi*t + phi)*u
    else:
        Fpi = 0.0*u

    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))

    rhsx = NL[...,0] + visc[...,0] + Fpi[...,0]
    rhsy = NL[...,1] + visc[...,1] + Fpi[...,1]
    rhsx = np.fft.rfftn(rhsx, axes=(0,1))
    rhsy = np.fft.rfftn(rhsy, axes=(0,1))

    if augmented:
        Z = Z_window_k(uhx.shape)
        uhx_new = uhx + dt*(rhsx - (gamma0*Z)*uhx - lam_e*np.exp(-alpha_e*t)*uhx)
        uhy_new = uhy + dt*(rhsy - (gamma0*Z)*uhy - lam_e*np.exp(-alpha_e*t)*uhy)
    else:
        uhx_new = uhx + dt*rhsx
        uhy_new = uhy + dt*rhsy

    ux = np.fft.irfftn(uhx_new, s=(Nx,Ny), axes=(0,1))
    uy = np.fft.irfftn(uhy_new, s=(Nx,Ny), axes=(0,1))
    unew = np.stack([ux, uy], axis=-1)
    return project_div_free(unew)

# Initial condition: Taylor–Green–like
u0 = np.zeros((Nx,Ny,2))
u0[...,0] =  np.sin(X)*np.cos(Y)
u0[...,1] = -np.cos(X)*np.sin(Y)
u0 = project_div_free(u0)

# Time loop params
dt = 2.5e-3
t_end = 4.0
nt = int(t_end/dt)

def run_case(augmented=True):
    u = u0.copy()
    T,E,Om,W = [],[],[],[]
    t=0.0
    for n in range(nt):
        u = step(u, t, dt, augmented=augmented)
        t += dt
        if n % 20 == 0:
            T.append(t); E.append(energy(u)); Om.append(enstrophy(u)); W.append(wmax(u))
    return np.array(T), np.array(E), np.array(Om), np.array(W)

print('Running classical...')
Tc, Ec, Omc, Wc = run_case(augmented=False)
print('Running NMSI (π*, γ_diss, e*)...')
Ta, Ea, Oma, Wa = run_case(augmented=True)

# Save
np.savez('tg_classical_timeseries.npz', t=Tc, E=Ec, Om=Omc, W=Wc)
np.savez('tg_aug_timeseries.npz',       t=Ta, E=Ea, Om=Oma, W=Wa)
print('Saved NPZ files.')

# Plots
plt.figure(); plt.plot(Tc,Ec,'--',label='Classical'); plt.plot(Ta,Ea,label='NMSI'); plt.xlabel('t'); plt.ylabel('Energy'); plt.legend(); plt.tight_layout(); plt.show()
plt.figure(); plt.plot(Tc,Omc,'--',label='Classical'); plt.plot(Ta,Oma,label='NMSI'); plt.xlabel('t'); plt.ylabel('Enstrophy'); plt.legend(); plt.tight_layout(); plt.show()
plt.figure(); plt.plot(Tc,Wc,'--',label='Classical'); plt.plot(Ta,Wa,label='NMSI'); plt.xlabel('t'); plt.ylabel('max|ω|'); plt.legend(); plt.tight_layout(); plt.show()
print('Final classical: E={:.3e}, Ω={:.3e}, max|ω|={:.3e}'.format(Ec[-1],Omc[-1],Wc[-1]))
print('Final NMSI:      E={:.3e}, Ω={:.3e}, max|ω|={:.3e}'.format(Ea[-1],Oma[-1],Wa[-1]))
